# Exercise 3

This sheet is about [density matrices](../QI_course/2_The_Qubit.pdf), [the partial trace](../QI_course/3_Quantum_Information.pdf) and [the Schmidt decomposition](../QI_course/6_Quantum_Correlations_part_1.pdf). Take a look at the linked materials to find out more.

## 1

Given an abitrary single qubit state $a|0\rangle +b|1\rangle$:

(a) Write down the correponding density matrix.

(b) Write down the density matrix representing the effect of applying an $X$ with probability $q_x$, $Y$ with probability $q_y$ and $Z$ with probability $q_z$.

(c) Write down the density matrix for representing the effect of replacing the state with $I/2$ with probability $p$.

(d) For what values of $q_x$, $q_y$ and $q_z$ is the effect of (b) equivalent to that of (c)?

## 2

(a) Write down the density matrix of a single qubit which is in state $|0\rangle$ with probability $2/3$ and $|+\rangle$ with probability $1/3$.

(b) Find a two qubit state such that one of the reduced density matrices is equal to the density matrix in (a).

(c) Write down a density matrix for a state that is not entangled, but which has the same reduced density matrices as your answer for (b).

## 3

Consider the three qubit state $(|001\rangle + |010\rangle)+ |100\rangle/\sqrt{3}$.

(a) What is the reduced density matrix for a single qubit in this state?

(b) What is the reduced density matrix for a pair of qubits in this state?

(c) Use the Schmidt decomposition to rewrite the state, with one qubit on one side of the bipartition and two qubits on the other.